In [1]:
import os

os.chdir("../..")

import utils

In [2]:
outputfolder = os.getcwd() + '/output/'
datafolder = os.getcwd() + '/datasets/PTB-XL/'
sampling_rate = 100
task = 'priority'
experiment_name = 'exp4'

data, raw_labels = utils.load_dataset(datafolder, sampling_rate=sampling_rate)
labels = utils.compute_label_aggregations(raw_labels, datafolder, task)
data, labels, Y, _ = utils.select_data(data, labels, task, 0, outputfolder+experiment_name+'/data/')

In [3]:
X_test = data[labels.strat_fold == 10]
y_test = Y[labels.strat_fold == 10]

X_val = data[labels.strat_fold == 9]
y_val = Y[labels.strat_fold == 9]

X_train = data[labels.strat_fold <= 8]
y_train = Y[labels.strat_fold <= 8]

n_classes = y_train.shape[1]

print(f"This experiment has {n_classes} classes")

X_train = X_train[:,:,0]
X_test = X_test[:,:,0]
X_val = X_val[:,:,0]

This experiment has 43 classes


In [8]:
import tensorflow as tf
from tensorflow.keras.layers import LSTM, GRU, TimeDistributed, Bidirectional, LeakyReLU
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten,  Input, Reshape
from tensorflow.keras.layers import Conv1D, MaxPool1D, GlobalAveragePooling1D,MaxPool1D,AveragePooling1D
from tensorflow.keras import initializers, regularizers, constraints
from tensorflow.keras.layers import Layer
import numpy as np
from tensorflow.keras.layers import BatchNormalization, ReLU, MaxPooling1D, LSTM
from tensorflow.keras import regularizers
from tensorflow.keras import backend as K

from tensorflow.keras.models import Model, Sequential


In [23]:
inputs = Input(shape=(1000,1))
x = Conv1D(filters=64, kernel_size=5, strides=1, padding='same')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)

x = Conv1D(filters=128, kernel_size=5, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)

x = Conv1D(filters=256, kernel_size=5, strides=1, padding='same')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = MaxPooling1D(pool_size=4, strides=4)(x)
# x = GlobalAveragePooling1D()(x)

x = LSTM(128, return_sequences=True)(x)
x = Dropout(0.5)(x)

x = LSTM(64, return_sequences=True)(x)
x = GlobalAveragePooling1D()(x)
x = Dropout(0.5)(x)

x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)

outputs = Dense(43, activation='sigmoid')(x)

model = Model(inputs=inputs, outputs=outputs)

model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 1000, 1)]         0         
                                                                 
 conv1d_21 (Conv1D)          (None, 1000, 64)          384       
                                                                 
 batch_normalization_20 (Bat  (None, 1000, 64)         256       
 chNormalization)                                                
                                                                 
 re_lu_20 (ReLU)             (None, 1000, 64)          0         
                                                                 
 max_pooling1d_20 (MaxPoolin  (None, 250, 64)          0         
 g1D)                                                            
                                                                 
 conv1d_22 (Conv1D)          (None, 250, 128)          4108

In [24]:
model.compile(
    loss=tf.keras.losses.BinaryCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
    metrics=[tf.keras.metrics.BinaryAccuracy(name='accuracy',threshold=0.5),
            tf.keras.metrics.Recall(name='recall'),
            tf.keras.metrics.AUC(num_thresholds=200,
                                curve="ROC",
                                summation_method='interpolation',
                                name="AUC",
                                multi_label=True,
                                label_weights=None)]
)

In [25]:
history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=30)

Epoch 1/30
534/534 [==============================] - 93s 157ms/step - loss: 0.1262 - accuracy: 0.9600 - recall: 0.4203 - AUC: 0.5418 - val_loss: 0.1125 - val_accuracy: 0.9629 - val_recall: 0.3947 - val_AUC: 0.6376
Epoch 2/30
534/534 [==============================] - 77s 144ms/step - loss: 0.1107 - accuracy: 0.9654 - recall: 0.4906 - AUC: 0.6235 - val_loss: 0.1042 - val_accuracy: 0.9666 - val_recall: 0.5213 - val_AUC: 0.6723
Epoch 3/30
534/534 [==============================] - 82s 154ms/step - loss: 0.1080 - accuracy: 0.9657 - recall: 0.4992 - AUC: 0.6512 - val_loss: 0.1061 - val_accuracy: 0.9666 - val_recall: 0.5178 - val_AUC: 0.6537
Epoch 4/30
534/534 [==============================] - 80s 149ms/step - loss: 0.1075 - accuracy: 0.9659 - recall: 0.5015 - AUC: 0.6538 - val_loss: 0.1078 - val_accuracy: 0.9654 - val_recall: 0.4589 - val_AUC: 0.6430
Epoch 5/30
534/534 [==============================] - 87s 163ms/step - loss: 0.1067 - accuracy: 0.9660 - recall: 0.5051 - AUC: 0.6576 - val_

In [18]:
from keras.utils import plot_model

# plot_model(model)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 1000, 1)]         0         
                                                                 
 conv1d_6 (Conv1D)           (None, 1000, 64)          384       
                                                                 
 batch_normalization_5 (Batc  (None, 1000, 64)         256       
 hNormalization)                                                 
                                                                 
 re_lu_5 (ReLU)              (None, 1000, 64)          0         
                                                                 
 max_pooling1d_5 (MaxPooling  (None, 250, 64)          0         
 1D)                                                             
                                                                 
 conv1d_7 (Conv1D)           (None, 250, 128)          41088 

In [26]:
from sklearn.metrics import roc_auc_score

preds = model.predict(X_test)

print(roc_auc_score(y_test,preds))
auc = model.evaluate(X_test, y_test)
print(auc)

68/68 [==============================] - 5s 42ms/step
0.7470347031178826
68/68 [==============================] - 4s 55ms/step - loss: 0.1060 - accuracy: 0.9624 - recall: 0.3508 - AUC: 0.6794
[0.10596800595521927, 0.9624312520027161, 0.35084426403045654, 0.6794029474258423]
